In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import xml.etree.ElementTree as ET
import pickle
import math
import copy
from rdp import *
token = "pk.eyJ1IjoibG1hZ25hbmEiLCJhIjoiY2s2N3hmNzgwMGNnODNqcGJ1N2l2ZXZpdiJ9.-aOxDLM8KbEQnJfXegtl7A"
px.set_mapbox_access_token(token)
n_voxel = 3
vox_divider = 2
nb_subvox = (10/vox_divider)

In [2]:
def load_gpx():
    with open('gpx.df','rb') as infile:
        df = pickle.load(infile)
    begin = int(df.iloc[-1]["route_num"])
    print(begin)
    for i in range(begin+1, begin+1109+1):
        tree = ET.parse('Datas/GPS/GPX/data'+str(i)+'.gpx')
        if(len(tree.getroot()) > 1):
            root = tree.getroot()[1][0]
            df_temp = pd.DataFrame(columns=['lat', 'lon'])
            j=0
            for child in root:
                coord = child.attrib
                coord['lat'] = float(coord['lat'])
                coord['lon'] = float(coord['lon'])
                df_temp = df_temp.append(pd.DataFrame(coord, index=[j]))
                j+=1
            df_temp["route_num"] = i
            df = df.append(df_temp)
    with open('gpx.df', 'wb') as outfile:
        pickle.dump(df, outfile)

In [3]:
def rd_compression(df, nb_routes=1110, eps=1e-4):
    """
    Compress a dataframe with douglas-peucker's algorithm.

    Parameters
    ----------
    df : pandas' DataFrame with columns=['lat', 'lon', 'route_num']
        Dataframe to compress
    eps : int in [0, 1[ , optional
        Precision of the compression (high value = few points)
    nb_routes : int
        Number of routes to compress

    Returns
    -------
    pandas' DataFrame with columns=['lat', 'lon', 'route_num']
        the compressed DataFrame
    """
    
    df_simplified = pd.DataFrame(columns=['lat', 'lon', 'route_num'])
    for i in range(1, nb_routes):
        route = df[df['route_num']==i].values
        if(len(route)>0):
            simplified = rdp(np.delete(route, 2, 1), epsilon=eps)
            simplified = np.insert(simplified, 2, route[0][2], axis=1) #add the route_number to the compressed route
            df_temp = pd.DataFrame(simplified, columns=['lat', 'lon', 'route_num'])
            df_simplified = df_simplified.append(df_temp)
    return df_simplified

In [4]:
def display(dfdisplay, n=75, line_group="route_num", color=None):
    """
    Display a dataframe of gps points on a mapbox map.
    Parameters
    ----------
    df or str : pandas' DataFrame with columns=['lat', 'lon', 'route_num'] or the name of a file containing one
        Dataframe to display or the file where it is located
    n : int, optional
        Number of routes to display
    line_group : str, optional
        Dataframe's attribute used to differenciate routes
    color : str, optional
        Dataframe's attribute used to color routes
    """
    n+=1
    if(type(dfdisplay) == str): #if df is a file location
        with open(dfdisplay,'rb') as infile:
            dfdisplay = pickle.load(infile) #open the file to load the dataframe
    dfdisplay = dfdisplay[dfdisplay[line_group]<n]
    fig = px.line_mapbox(dfdisplay, lat="lat", lon="lon", line_group=line_group, color=color, zoom=11)
    fig.show()

In [5]:
#display("gpx_simplified.df", 1)

In [6]:
import math
def truncate(number, digits) -> float:
    stepper = 10.0 ** digits
    return math.trunc(stepper * number) / stepper

In [7]:
def find_voxel_int(p):
    """
    Find the voxel in which a point is by truncating its position. Voxel's position are transformed into 
    int to be manipulated in an easier way.
    Parameters
    ----------
    p : list of two int
        The point 
    n : int, optional
        Number of digits to truncate
        
    Returns
    -------
    list of two int
        Position of the voxel's low left point
    """
    v_lat = math.trunc(p[0]*10**(n_voxel+1))
    v_lon = math.trunc(p[1]*10**(n_voxel+1))
    
    while(v_lat%nb_subvox != 0):
        v_lat -= 1
    while(v_lon%nb_subvox != 0):
        v_lon -= 1
    
    return [v_lat, v_lon]

In [8]:
def line_intersection(line1, line2):
    """
    Find the point of intersection between two lines
    Parameters
    ----------
    line1 : list of two points (a point is a list of two int)
        First line  
    line2 : list of two points (a point is a list of two int)
        Second line  
        
    Returns
    -------
    list of two int
        Position of the intersection
    """
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
       raise Exception('lines do not intersect')

    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return [x, y]
    

In [9]:

def create_dict_vox(df, nb_routes):
    """
    With a dataframe containing gps points separated in routes, creates a dict of voxels.  
    Parameters
    ----------
     df : pandas' DataFrame with columns=['lat', 'lon', 'route_num']
        Dataframe to use 
    nb_routes : int
        Number of routes to use in the dataframe 
        
    Returns
    -------
    dict of voxels
        Keys of this dict are strings containing the position of voxels' low left points transformed to int
        and separated by a ';'.
        Values of this dict are lists containing the number of all routes that pass through the voxel.
    """
    
    dict_vox = {}
    
    for r in range(1, nb_routes+1):
        
        route = df[df["route_num"]==r]
        points = route.values.tolist()

        for j in range(len(points)-1):
            p1 = points[j] #we take two points in the dataframe that create a line
            p2 = points[j+1]

            if(p1[0]>p2[0]):
                lat_orientation = -nb_subvox #the line is going down
            else:
                lat_orientation = nb_subvox #the line is going up

            if(p1[1]>p2[1]):
                lon_orientation = -nb_subvox #the line is going left 
            else:
                lon_orientation = nb_subvox #the line is goin right

            vox_int = find_voxel_int(p1) #find the start voxel
            vox_final_int = find_voxel_int(p2) #find the final voxel
            
            
            
            #while the final voxel has not been reached
            while(vox_int[0] != vox_final_int[0] or vox_int[1] != vox_final_int[1]):
                vox_float = [vox_int[0]*10**(-n_voxel-1), vox_int[1]*10**(-n_voxel-1)] #transform the vox into real points
                
                key = str(int(vox_int[0]))+";"+str(int(vox_int[1])) #save the voxel
                if key in dict_vox:
                    if(r not in dict_vox[key]):
                        dict_vox[key].append(r)
                else :
                    dict_vox[key] = [r]
                    
                '''find the good intersection point (if the line is going up, we search the intersection between 
                it and the up line of the voxel for example)'''
                if(lat_orientation>0):
                    intersection_lat = line_intersection([p1, p2], [[vox_float[0]+nb_subvox*10**(-n_voxel-1), vox_float[1]],
                                                        [vox_float[0]+nb_subvox*10**(-n_voxel-1), vox_float[1]+nb_subvox*10**(-n_voxel-1)]])
                else:
                    intersection_lat = line_intersection([p1, p2], [vox_float, [vox_float[0], vox_float[1]+nb_subvox*10**(-n_voxel-1)]])

                    
                '''same for left and right'''
                if(lon_orientation>0): 
                    intersection_lon = line_intersection([p1, p2], [[vox_float[0], vox_float[1]+nb_subvox*10**(-n_voxel-1)], 
                                                        [vox_float[0]+nb_subvox*10**(-n_voxel-1), vox_float[1]+nb_subvox*10**(-n_voxel-1)]])
                else:
                    intersection_lon = line_intersection([p1, p2], [vox_float, [vox_float[0]+nb_subvox*10**(-n_voxel-1), vox_float[1]]])

                #calculate the distance between the first point of the line and the two intersection points
                intersection_lon_distance = sqrt((p1[0]-intersection_lon[0])**2+(p1[1]-intersection_lon[1])**2)
                intersection_lat_distance = sqrt((p1[0]-intersection_lat[0])**2+(p1[1]-intersection_lat[1])**2)

                #find the shorter distance then go to the next voxel using the orientation of the line
                if(intersection_lat_distance<intersection_lon_distance): 
                    vox_int[0] += lat_orientation
                else:
                    vox_int[1] += lon_orientation
                    
            key = str(int(vox_int[0]))+";"+str(int(vox_int[1])) #end of the while loop, save the last voxel
            if key in dict_vox:
                if(r not in dict_vox[key]):
                    dict_vox[key].append(r)
            else :
                 dict_vox[key] = [r]
    return dict_vox


    


In [10]:
def get_voxel_points(vox, num_vox):
    """
    Take the position of the low left point of a voxel transformed into an int 
    and return this voxel's four real points.
    Parameters
    ----------
    vox : list of two int
        Position of the voxel's low left point transformed into an int
    num_vox : int
        Number of the voxel, used later to differentiate voxels
        
    Returns
    -------
    list 
        list of the four points (a point is a list of two int)
    """
    tab_vox = []
    vox_float = [vox[0]*10**(-n_voxel-1), vox[1]*10**(-n_voxel-1)]
    vox_float.append(num_vox)
    vox_float.append(1)
    tab_vox.append(vox_float)
    tab_vox.append([vox_float[0]+nb_subvox*10**(-n_voxel-1), vox_float[1], num_vox, 1])
    tab_vox.append([vox_float[0]+nb_subvox*10**(-n_voxel-1), vox_float[1]+nb_subvox*10**(-n_voxel-1), num_vox, 1])
    tab_vox.append([vox_float[0], vox_float[1]+nb_subvox*10**(-n_voxel-1), num_vox, 1])
    return tab_vox


In [11]:
def get_adjacent_voxel(vox, lat_diff, lon_diff):
    return [vox[0]+lat_diff*nb_subvox, vox[1]+lon_diff*nb_subvox]

In [12]:
def voxel_convolution(vox, dict_vox, dict_vox_used, num_vox, lat_diff, lon_diff):
    """
    With a voxel, check if one of his neighbour exists and if it has already been used.
    ----------
    vox : list of two int
        Position of the voxel's low left point transformed into an int
    dict_vox : dict
        Dictionary of existing voxels 
    dict_vox_used : dict
        Dictionary of voxels that have already been used
    num_vox : int
        Number of the voxel, used later to differentiate voxels
    lat_diff : int
        Difference of latitude (the unit is voxel) between the voxel and the neighbour
    lon_diff : int
        Difference of longitude (the unit is voxel) between the voxel and the neighbour
        
    Returns
    -------
    list
        If the voxel exists and has not been used : 
            A list containing the voxel's low left point transformed into an int and the list containing all routes
            that are going through the voxel
        Else:
            An empty list
    """
    vox_adj = get_adjacent_voxel(vox, lat_diff, lon_diff)
    key_adj = str(int(vox_adj[0]))+";"+str(int(vox_adj[1]))
    if(key_adj in dict_vox and not(key_adj in dict_vox_used)):
        return [vox_adj, dict_vox[key_adj], key_adj]
    return []
        
    

In [13]:
def get_voxels_with_min_routes(dict_vox, min_routes):
    """
    Return all voxels or groups of voxels that have at least a number of routes passing through themselves.
    Parameters
    ----------
    dict_vox : dict
        Dictionary of existing voxels 
    min_routes : int
        Minimum number of routes passing through voxels / groups of voxels
        
    Returns
    -------
    list 
        List of voxels that have or are part of a group that have at least 'min_routes' routes 
        passing through itself. A voxel is a list of four points.
    """
    num_vox = 0 #used to differentiate voxels
    dict_vox_used = {}
    tab_voxel_with_min_routes = [] #final list containing all voxels that matches with the conditions
    
    for key in dict_vox: #for all vosels
        tab_routes = dict_vox[key]
        
        vox_str = key.split(";")
        vox_int = [int(vox_str[0]), int(vox_str[1])]
        
        
        #if the voxels has at least 'min_routes' routes and has not been saved we save it
        if(key not in dict_vox_used and len(tab_routes) >= min_routes):
            tab_voxel_with_min_routes += get_voxel_points(vox_int, num_vox)
            dict_vox_used[key] = True
            num_vox -= 1
            
        
        #creation of a list containing all neighbours of the voxel
        tab_vox_adj = []
        tab_vox_adj.append(voxel_convolution(vox_int, dict_vox, dict_vox_used, num_vox, -1, 0))
        tab_vox_adj.append(voxel_convolution(vox_int, dict_vox, dict_vox_used, num_vox, 1, 0))
        tab_vox_adj.append(voxel_convolution(vox_int, dict_vox, dict_vox_used, num_vox, 0, 1))
        tab_vox_adj.append(voxel_convolution(vox_int, dict_vox, dict_vox_used, num_vox, 0, -1))
        
        #for all neighbours
        for vox_adj in tab_vox_adj:
            if(len(vox_adj) != 0):#if it exists and has not been used 
                
                union_tab_routes = list(set(tab_routes) | set(vox_adj[1]))#union of lists
                
                if(len(union_tab_routes) >= min_routes):
                    key_adj = str(int(vox_adj[0][0]))+";"+str(int(vox_adj[0][1]))
                    if(len(union_tab_routes) > len(tab_routes) and key_adj not in dict_vox_used):
                        tab_voxel_with_min_routes += get_voxel_points(vox_adj[0], num_vox)
                        dict_vox_used[key_adj] = True
                        num_vox -= 1
    return tab_voxel_with_min_routes


In [17]:
with open("gpx_simplified.df",'rb') as infile:
        df_simplified = pickle.load(infile)
               
nb_routes = 1
min_routes = 2

df_simplified["type"] = 0
df_display = df_simplified[(df_simplified["route_num"]<=nb_routes)]

dict_voxels = create_dict_vox(df_display, nb_routes)
tab_vox = get_voxels_with_min_routes(dict_voxels, min_routes)

        
df = pd.DataFrame(tab_vox, columns=["lat", "lon", "route_num", "type"])
df_display = df_display.append(df)
display(df_display, color="type")    


In [15]:
def get_similitude(dict_voxels, num_route1, num_route2):
    t = []
    t.append([])
    t.append([])
    for key in dict_voxels:
        tab_routes = dict_voxels[key]
        if(num_route1 in dict_voxels[key]):
            t[0].append(key)
        if(num_route2 in dict_voxels[key]):
            t[1].append(key)

        vox_str = key.split(";")
        vox_int = [int(vox_str[0]), int(vox_str[1])]
        tab_vox_adj = []
        tab_vox_adj.append(voxel_convolution(vox_int, dict_voxels, {}, 0, -1, 0))
        tab_vox_adj.append(voxel_convolution(vox_int, dict_voxels, {}, 0, 1, 0))
        tab_vox_adj.append(voxel_convolution(vox_int, dict_voxels, {}, 0, 0, 1))
        tab_vox_adj.append(voxel_convolution(vox_int, dict_voxels, {}, 0, 0, -1))
        for vox in tab_vox_adj:
            if(len(vox)>0):
                union_tab_routes = list(set(tab_routes) | set(vox[1]))#union of lists
                key_adj = vox[2]
                if(len(union_tab_routes) > len(tab_routes) and len(union_tab_routes) > len(vox[1])):
                    if(num_route1 in dict_voxels[key_adj]):
                        t[0].append(key)
                        break
                    if(num_route2 in dict_voxels[key_adj]):
                        t[1].append(key)
                        break
    common_parts = len(list(set(t[0]) & set(t[1])))
    #print(common_parts, len(t[0]), len(t[1]))
    return [common_parts/len(t[0]), common_parts/len(t[1])]

In [16]:
with open("gpx_simplified.df",'rb') as infile:
        df_cluster = pickle.load(infile)
        
dict_voxels = create_dict_vox(df_display, df_cluster.iloc[-1]["route_num"])
print(get_similitude(dict_voxels, 1, 2))


[0.054474708171206226, 0.08045977011494253]
